In [1]:
import os
rootDir = 'D:\\RJ\\UCI\\Ralph School\\2023 Spring\\CS 121\\Assignments\\.vscode\\Res\\Dev'
jsonFiles = []
for root, dirs, files in os.walk(rootDir):
    for name in files:
        if name.endswith((".json")):
            full_path = os.path.join(root, name)
            jsonFiles.append(full_path)

jsonFiles = jsonFiles[:10]

In [2]:
import json
import shelve
from HelperClass import InvertedIndex, Token
from helper import tokenizeHtml

In [3]:
# Opening JSON file
def getJsonData(filePath):
    jFile = open(filePath)
    data = json.load(jFile)
    url = data['url']
    htmlContent = data['content']
    jFile.close()
    return (url, htmlContent)

In [4]:
invIndex = InvertedIndex() #Create inverted index to hold tokens from parser
docId = 0
for jFile in jsonFiles:
    docId += 1
    url, htmlContent = getJsonData(jFile)
    # Cleans and parses HTML content into tokens then adds it to Inverted index
    tokenizeHtml(docId=docId, invIndex=invIndex, htmlContent=htmlContent)
    print(docId, url)

1 https://aiclub.ics.uci.edu/
2 https://aiclub.ics.uci.edu/#body
3 https://aiclub.ics.uci.edu/index.html
4 http://alderis.ics.uci.edu/links.html
5 http://alderis.ics.uci.edu/publications.html
6 http://alderis.ics.uci.edu/
7 http://alderis.ics.uci.edu/dresystems.html
8 http://alderis.ics.uci.edu/amba2.html
9 http://alderis.ics.uci.edu/index.html
10 http://alderis.ics.uci.edu/downloads.html


In [5]:
invIndex

Total Tokens: 627
Token: artifici
	DocId: 1, Freq: 5
		Pos: {384, 1, 46, 211, 21}
		Weights: {'title': {1}, 'h3': {21, 46}, 'normal': {211}, 'footer': {384}}
	DocId: 2, Freq: 5
		Pos: {384, 1, 46, 211, 21}
		Weights: {'title': {1}, 'h3': {21, 46}, 'normal': {211}, 'footer': {384}}
	DocId: 3, Freq: 5
		Pos: {384, 1, 46, 211, 21}
		Weights: {'title': {1}, 'h3': {21, 46}, 'normal': {211}, 'footer': {384}}

Token: intellig
	DocId: 1, Freq: 5
		Pos: {385, 2, 47, 212, 22}
		Weights: {'title': {2}, 'h3': {22, 47}, 'normal': {212}, 'footer': {385}}
	DocId: 2, Freq: 5
		Pos: {385, 2, 47, 212, 22}
		Weights: {'title': {2}, 'h3': {22, 47}, 'normal': {212}, 'footer': {385}}
	DocId: 3, Freq: 5
		Pos: {385, 2, 47, 212, 22}
		Weights: {'title': {2}, 'h3': {22, 47}, 'normal': {212}, 'footer': {385}}

Token: uci
	DocId: 1, Freq: 6
		Pos: {386, 3, 197, 20, 56, 30}
		Weights: {'title': {3}, 'h1': {20}, 'h3': {56, 30}, 'normal': {197}, 'footer': {386}}
	DocId: 2, Freq: 6
		Pos: {386, 3, 197, 20, 56, 30}
	

In [33]:
import pandas as pd

def df_from_dict(token:Token):
    df = pd.DataFrame.from_dict(token.getAllPos(), orient='index')
    df.fillna(0, inplace=True)
    df = df.astype('int32')
    return df.transpose()

#Cite: https://stackoverflow.com/questions/47545052/convert-dataframe-rows-to-python-set
def dict_from_df(df) -> dict:
    df = df.transpose()
    series_set = df.apply(frozenset, axis=1)
    new_df = series_set.apply(lambda a: set(a))
    return dict(new_df)

def join_df_col(df1,df2):
    df3 = df1.join(df2)
    df3.fillna(0, inplace=True)
    return df3.astype('int32')

df = df_from_dict(invIndex['artifici'])
print(df)

     1    2    3
0  384  384  384
1    1    1    1
2   46   46   46
3  211  211  211
4   21   21   21


In [6]:
invIndex['artifici'].getAllPos()

{1: {1, 21, 46, 211, 384}, 2: {1, 21, 46, 211, 384}, 3: {1, 21, 46, 211, 384}}

In [26]:
fTok = Token('artifici')
fTok.addDoc(1)
fTok.addPosition(1,21)
fTok.addPosition(1,46)
fTok.addPosition(1,211)
fTok.addPosition(1,384)
print(fTok.getAllPos())

fTok2 = Token('artifici')
fTok2.addDoc(2)
fTok2.addPosition(2,21)
fTok2.addPosition(2,46)
fTok2.addPosition(2,211)
fTok2.getAllPos()

{1: {384, 211, 21, 46}}


{2: {21, 46, 211}}

In [27]:
print(df_from_dict(fTok))

     1
0  384
1  211
2   21
3   46


In [28]:
print(df_from_dict(fTok2))

     2
0  211
1   21
2   46


     1    2
0  384  211
1  211   21
2   21   46
3   46    0
